In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [38]:
train=pd.read_csv("/Users/sudhanvasavyasachi/Desktop/Projects/Credit Card Default/Data/processed_train.csv")

In [39]:
test=pd.read_csv("/Users/sudhanvasavyasachi/Desktop/Projects/Credit Card Default/Data/processed_test.csv")

In [40]:
train.columns

Index(['marriage', 'sex', 'education', 'limit_bal', 'age', 'pay_1', 'pay_2',
       'pay_3', 'pay_4', 'pay_5', 'pay_6', 'bill_amt1', 'bill_amt2',
       'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6', 'pay_amt1',
       'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6',
       'pay_to_bill_ratio', 'next_month_default', 'avg_bill_amt'],
      dtype='object')

In [41]:
test.columns

Index(['marriage', 'sex', 'education', 'limit_bal', 'age', 'pay_1', 'pay_2',
       'pay_3', 'pay_4', 'pay_5', 'pay_6', 'bill_amt1', 'bill_amt2',
       'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6', 'pay_amt1',
       'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6',
       'pay_to_bill_ratio', 'next_month_default', 'avg_bill_amt'],
      dtype='object')

# Feature Addition

In [42]:
train['total_pay'] = train[[f'pay_amt{i}' for i in range(1, 7)]].apply(lambda x: x.sum(), axis=1)
train['total_bill'] = train[[f'bill_amt{i}' for i in range(1, 7)]].apply(lambda x: x.sum(), axis=1)
test['total_pay'] = test[[f'pay_amt{i}' for i in range(1, 7)]].apply(lambda x: x.sum(), axis=1)
test['total_bill'] = test[[f'bill_amt{i}' for i in range(1, 7)]].apply(lambda x: x.sum(), axis=1)

In [43]:
train['average_utilization']=train['avg_bill_amt']/train['limit_bal']
test['average_utilization']=test['avg_bill_amt']/test['limit_bal']

In [44]:
delay_cols = ['pay_1', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']
train['delinquency_score'] = train[delay_cols].sum(axis=1)
test['delinquency_score'] = test[delay_cols].sum(axis=1)

In [45]:
pay_cols = ['pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']
train['consistency_score'] = train[pay_cols].std(axis=1)

In [46]:
test['consistency_score'] = test[pay_cols].std(axis=1)

In [47]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22086 entries, 0 to 22085
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   marriage             22086 non-null  int64  
 1   sex                  22086 non-null  int64  
 2   education            22086 non-null  int64  
 3   limit_bal            22086 non-null  int64  
 4   age                  21970 non-null  float64
 5   pay_1                22086 non-null  int64  
 6   pay_2                22086 non-null  int64  
 7   pay_3                22086 non-null  int64  
 8   pay_4                22086 non-null  int64  
 9   pay_5                22086 non-null  int64  
 10  pay_6                22086 non-null  int64  
 11  bill_amt1            22086 non-null  float64
 12  bill_amt2            22086 non-null  float64
 13  bill_amt3            22086 non-null  float64
 14  bill_amt4            22086 non-null  float64
 15  bill_amt5            22086 non-null 

## Dropping the null values from both train and test

In [48]:
train.dropna(inplace=True)
test.dropna(inplace=True)

## Mutual information

In [50]:
# calculate the MI score for every feature in train set wrt to target variable
from sklearn.feature_selection import mutual_info_classif
def calculate_mi_scores(X, y):
    mi_scores = mutual_info_classif(X, y, discrete_features='auto', random_state=42)
    mi_scores_df = pd.DataFrame(mi_scores, index=X.columns, columns=['MI Score'])
    return mi_scores_df.sort_values(by='MI Score', ascending=False)
X_train = train.drop(columns=['next_month_default'])
y_train = train['next_month_default']
mi_scores_train = calculate_mi_scores(X_train, y_train)
# print the MI scores
print(mi_scores_train)

                     MI Score
pay_1                0.071571
delinquency_score    0.064369
pay_2                0.043221
pay_4                0.035083
pay_3                0.032263
pay_5                0.026135
pay_6                0.025572
pay_amt1             0.024785
total_pay            0.019644
pay_amt3             0.017415
average_utilization  0.016164
pay_amt2             0.016035
pay_amt4             0.015921
avg_bill_amt         0.014903
pay_amt5             0.014355
pay_to_bill_ratio    0.014094
total_bill           0.014053
consistency_score    0.011577
limit_bal            0.010265
sex                  0.009206
pay_amt6             0.008819
bill_amt3            0.007998
education            0.005395
bill_amt6            0.004888
bill_amt4            0.004387
bill_amt2            0.004252
age                  0.003691
marriage             0.003523
bill_amt5            0.003518
bill_amt1            0.002874


In [ ]:
train.to_csv("engineered_train_1.csv",index=False)
test.to_csv("engineered_test_1.csv",index=False)